In [ ]:
import os
import cvxpy as cp
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import cv2
import torch

In [ ]:
# preprocess and load train images

transform1 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((150, 150)),
])


In [ ]:
# resize and get training data

df = pd.read_csv('train_small.csv')
path_to = "train_small/train_small"

X = []
y_vec = []

for idx in range(len(df.index)):
    img_name = df.iloc[idx, 1]

    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

    image_path = os.path.join(path_to, img_name)
    abs_image_path = os.path.abspath(image_path)
    
    img_pil = Image.open(abs_image_path).convert('RGB')
    gray = img_pil.convert('L')
    gray = np.array(gray)
    img = np.array(img_pil, dtype=np.float32) / 255.0
    
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    for (x, y, w, h) in faces:
        if w > 100 and h > 100:
            img = img[y:y+h, x:x+w]
        break
    
    img = transform1(img)
    label = df.iloc[idx, 2]
    X.append(img)
    y_vec.append(label)

In [ ]:
rgb = np.vstack([image.reshape(-1, 3) for image in X])

In [ ]:
total_mean = np.mean(rgb, axis=0)
total_std = np.std(rgb, axis=0)

transform2 = transforms.Compose([
    transforms.Normalize(mean=total_mean, std=total_std),
])

In [ ]:
# create test train splits and dataset (for train small)
import torch.nn as nn
import torch.nn.functional as F
label_encoder = LabelEncoder()

y_vec = label_encoder.fit_transform(y_vec)

X_tensor = torch.stack(X)
total_mean = torch.tensor(total_mean).view(-1,1,1)
total_std = torch.tensor(total_std).view(-1,1,1)
X_normal = (X_tensor - total_mean) / total_std

y_tensor = torch.tensor(y_vec, dtype=torch.long)

X_train, X_test, y_train, y_test = train_test_split(X_normal, y_tensor, test_size=0.33, random_state=42)

class ArrayDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        feature = self.features[idx]
        label = self.labels[idx]
        return feature, label
    
train_dataset = ArrayDataset(X_train, y_train)
test_dataset = ArrayDataset(X_test, y_test)

batch_size = 64 

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
#create neural network
class CelebrityCNN(nn.Module):
    def __init__(self): # 100 classes
        super(CelebrityCNN, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=5, padding=2)
        
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5, padding=2)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=5, padding=1)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=5, padding=2)
        self.bn2 = nn.BatchNorm2d(256)
        
        self.fc1 = nn.Linear(256 * 8 * 8, 1024)
        self.fc2 = nn.Linear(1024, 100) 
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.bn1(self.conv2(x))))

        x = self.pool(F.relu(self.conv3(x)))

        x = self.pool(F.relu(self.bn2(self.conv4(x))))
        
        x = x.view(-1, 256 * 8 * 8)
        
        x = F.relu(self.fc1(x))
        x = self.fc2(x) 
        
        return x

def train(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        run_loss = 0.0
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            run_loss += loss.item()
        print(f'Epoch {epoch+1}, Loss: {run_loss/len(train_loader)}')
        
def evaluate(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy: {100 * correct / total}%')
    
num_classes = 100
model = CelebrityCNN()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
train(model, train_loader, criterion, optimizer, num_epochs=10)
evaluate(model, test_loader)

In [ ]:
df = pd.read_csv('train.csv')
path_to = "train/train"

X_train = []
y_train = []

for idx in range(len(df.index)):
    img_name = df.iloc[idx, 1]

    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

    image_path = os.path.join(path_to, img_name)
    abs_image_path = os.path.abspath(image_path)
    
    img_pil = Image.open(abs_image_path).convert('RGB')
    gray = img_pil.convert('L')
    gray = np.array(gray)
    img = np.array(img_pil, dtype=np.float32) / 255.0
    
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    for (x, y, w, h) in faces:
        if w > 100 and h > 100:
            img = img[y:y+h, x:x+w]
        break
    
    img = transform1(img)
    label = df.iloc[idx, 2]
    X_train.append(img)
    y_train.append(label)

In [ ]:
rgb = np.vstack([image.reshape(-1, 3) for image in X_train[0:10000]]) #take a sample for normalization

In [ ]:
total_mean = np.mean(rgb, axis=0)
total_std = np.std(rgb, axis=0)

In [ ]:
path_to = "test"
X_final_test = []

for idx in range(4977):
    img_name = str(idx) + ".jpg"
    

    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

    image_path = os.path.join(path_to, img_name)
    abs_image_path = os.path.abspath(image_path)
    
    img_pil = Image.open(abs_image_path).convert('RGB')
    gray = img_pil.convert('L')
    gray = np.array(gray)
    img = np.array(img_pil, dtype=np.float32) / 255.0
    
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    for (x, y, w, h) in faces:
        if w > 100 and h > 100:
            img = img[y:y+h, x:x+w]
        break
        
    img = transform1(img)
    X_final_test.append(img)


In [ ]:

transform2 = transforms.Compose([
    transforms.Normalize(mean=total_mean, std=total_std),
])

class ArrayDataset(Dataset):
    def __init__(self, features, labels, transform):
        self.features = features
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        feature = self.transform(self.features[idx])
        label = self.labels[idx]
        
        return feature, label
    
class TestDataset(Dataset):

    def __init__(self, features, transform):
        self.features = features
        self.transform = transform
        
    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        feature = self.transform(self.features[idx])

        return feature

In [ ]:
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)


y_tensor = torch.tensor(y_train, dtype=torch.long)

X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_train, test_size=0.33)

train_dataset = ArrayDataset(X_train, y_train, transform2)
test_dataset = ArrayDataset(X_test, y_test, transform2)

batch_size = 64 

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
model = CelebrityCNN()
train(model, train_loader, criterion, optimizer, num_epochs=10)
evaluate(model, test_loader)

In [ ]:
from torch.utils.data import TensorDataset

X_final_tensor = torch.stack(X_final_test)

final_test_dataset = TestDataset(X_final_tensor, transform=transform2)
test_dataloader = DataLoader(final_test_dataset, batch_size=64, shuffle=False)

all_outputs = torch.tensor([])
model.eval() 
with torch.no_grad():
    for batch in test_dataloader:
        
        outputs = model(batch)
        _, predicted = torch.max(outputs.data, 1)
        
        all_outputs = torch.cat((all_outputs, predicted), dim=0)
        

In [ ]:
all_outputs_np = all_outputs.numpy().astype(int)

df2 = pd.read_csv('category.csv')
labels = df2.iloc[:, 1]

label_encoder.fit(labels)

predicted_labels = label_encoder.inverse_transform(all_outputs_np)

id_num = range(len(predicted_labels))

out_data = pd.DataFrame({"Id": id_num, "Category": predicted_labels})

out_data.to_csv('submission2.csv', index=False)